In [ ]:
! pip install llama_index
! pip install transformers
! pip install langchain


In [ ]:
# need to install from github
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install datasets sentencepiece
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto"
                         )

In [ ]:
from transformers import  GenerationConfig, pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset

# Upload Training Material

Convert training materials into a format that can be accepted by the datasets library provided by Hugging Face. If file is a text file, use the load_dataset function in the datasets library to load dataset.

In [ ]:

train_data = load_dataset('text', data_files='/content/test.txt')


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=128)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b")

tokenized_datasets = train_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# 3. Initialize the model and the training arguments
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b")

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

In [ ]:
generate_text('Is Dhoni the best batsman of IPL?')

[{'generated_text': "Hardly.\n\nDhoni has a unique skill set that makes him one of the best Test players ever to don a cricket shirt but, historically, not the best in IPL.\n\nWhile Sachin Tendulkar and Ricky Ponting are considered the best batsmen to have played in IPL, it's still a 50-over format and there are some amazing players that played in IPL but were not considered the best in their positions.\n\nBelow is a list of the best Test batsman to have played in IPL but not the best:\n\nRohit Sharma\nSamiibhai Nadkarni\nGautam Ganbhir\nChris Green\nIshan Kishan\nVirat Kohli\nDhawal Kulkarni\nAjinkya Rahane\nImran Tahir"}]